In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import scipy
from tqdm.auto import tqdm, trange
import xgboost as xgb
import joblib
from sklearn.model_selection import cross_validate
import zipfile

from src.model import tscv, ClippedOutputRegressor
from src.feature_engineering import get_feature_cols, df_to_X_y, drop_non_features, add_lagged_features


%run constants.py

baseline_reg = joblib.load(os.path.join(MODELS_DIR, 'xgb-baseline.model'))

%matplotlib inline
print("Versions:")
print("  Python: %s" % sys.version)
for module in [pd, np, sns, sklearn]:
    print("  %s: %s" %(module.__name__, module.__version__))

Versions:
  Python: 3.8.2 (default, Jul 16 2020, 14:00:26) 
[GCC 9.3.0]
  pandas: 1.1.1
  numpy: 1.19.1
  seaborn: 0.10.1
  sklearn: 0.23.2


# Rank features

We can get the rank of a sample price sales count with relation to the others in a group.

We want, for each date block, the ranks in the following groups:

- item_id
- category
- shop + category

In [16]:
prices_stats = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'prices-statistics.parquet'))
train_set = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'train-set.parquet'))
categories_meta = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'item-categories-metadata.parquet'))

In [20]:
prices_stats = prices_stats.merge(categories_meta, on='item_id')
train_set = train_set.merge(categories_meta, on='item_id')

In [21]:
train_set.head()

,item_id,shop_id,date_block_num,item_cnt_month,item_category_id,category_name,subcategory_name
0,5037,5,10,0.0,19,Игры,PS3
1,5037,4,10,0.0,19,Игры,PS3
2,5037,6,10,0.0,19,Игры,PS3
3,5037,3,10,0.0,19,Игры,PS3
4,5037,2,10,0.0,19,Игры,PS3


In [22]:
prices_stats.head()

,item_id,shop_id,date_block_num,item_shop_price_median,cat_price_median,cat_shop_price_median,item_price_median,item_category_id,category_name,subcategory_name
0,5037,5,10,1699.0,1699.0,1999.0,1699.0,19,Игры,PS3
1,5037,4,10,1699.0,1699.0,1649.0,1699.0,19,Игры,PS3
2,5037,6,10,1699.0,1699.0,1999.0,1699.0,19,Игры,PS3
3,5037,3,10,1699.0,1699.0,1499.0,1699.0,19,Игры,PS3
4,5037,2,10,1699.0,1699.0,1499.0,1699.0,19,Игры,PS3


In [26]:
ranks_index_cols = [['item_id'],
                    ['category_name'],
                    ['shop_id', 'category_name']]

ranks_df = train_set.copy()
for idx_cols in tqdm(ranks_index_cols):
    price_rank_col = 'rank_%s_item_price_median' % ('_'.join(idx_cols))
    item_cnt_rank_col = 'rank_%s_item_cnt_month' % ('_'.join(idx_cols))
    
    ranks_df[price_rank_col] = prices_stats.groupby(idx_cols + ['date_block_num'])['item_price_median'].rank('dense')
    ranks_df[item_cnt_rank_col] = train_set.groupby(idx_cols + ['date_block_num'])['item_cnt_month'].rank('dense')

In [27]:
ranks_df.describe()

,item_id,shop_id,date_block_num,item_cnt_month,item_category_id,rank_item_id_item_price_median,rank_item_id_item_cnt_month,rank_category_name_item_price_median,rank_category_name_item_cnt_month,rank_shop_id_category_name_item_price_median,rank_shop_id_category_name_item_cnt_month
count,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5140800.0,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06
mean,1.101940e+04,3.164286e+01,2.150000e+01,2.199702e-01,4.630961e+01,1.0,1.194424e+00,2.764095e+01,1.218751e+00,2.764095e+01,1.199101e+00
std,6.252631e+03,1.756189e+01,6.922187e+00,1.113889e+00,1.671654e+01,0.0,8.550523e-01,1.954779e+01,1.096516e+00,1.954779e+01,8.842873e-01
min,3.000000e+01,2.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00,1.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
25%,5.381500e+03,1.600000e+01,1.575000e+01,0.000000e+00,3.700000e+01,1.0,1.000000e+00,1.300000e+01,1.000000e+00,1.300000e+01,1.000000e+00
50%,1.120300e+04,3.450000e+01,2.150000e+01,0.000000e+00,4.300000e+01,1.0,1.000000e+00,2.400000e+01,1.000000e+00,2.400000e+01,1.000000e+00
75%,1.607150e+04,4.700000e+01,2.725000e+01,0.000000e+00,5.800000e+01,1.0,1.000000e+00,3.900000e+01,1.000000e+00,3.900000e+01,1.000000e+00
max,2.216700e+04,5.900000e+01,3.300000e+01,2.000000e+01,8.300000e+01,1.0,2.100000e+01,1.440000e+02,2.100000e+01,1.440000e+02,2.100000e+01


And now that we have them for each month we can use them as lagged features:

In [28]:
train_set_ranks = add_lagged_features(train_set, ranks_df, [col for col in ranks_df if col.startswith('rank_')], max_lag=6)
train_set_ranks.describe()

,item_id,shop_id,date_block_num,item_cnt_month,item_category_id,f__rank_item_id_item_price_median_1,f__rank_item_id_item_cnt_month_1,f__rank_category_name_item_price_median_1,f__rank_category_name_item_cnt_month_1,f__rank_shop_id_category_name_item_price_median_1,...,f__rank_category_name_item_price_median_5,f__rank_category_name_item_cnt_month_5,f__rank_shop_id_category_name_item_price_median_5,f__rank_shop_id_category_name_item_cnt_month_5,f__rank_item_id_item_price_median_6,f__rank_item_id_item_cnt_month_6,f__rank_category_name_item_price_median_6,f__rank_category_name_item_cnt_month_6,f__rank_shop_id_category_name_item_price_median_6,f__rank_shop_id_category_name_item_cnt_month_6
count,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,4926600.0,4.926600e+06,4.926600e+06,4.926600e+06,4.926600e+06,...,4.069800e+06,4.069800e+06,4.069800e+06,4.069800e+06,3855600.0,3.855600e+06,3.855600e+06,3.855600e+06,3.855600e+06,3.855600e+06
mean,1.101940e+04,3.164286e+01,2.150000e+01,2.199702e-01,4.630961e+01,1.0,1.192897e+00,2.735358e+01,1.217200e+00,2.735358e+01,...,2.567271e+01,1.212778e+00,2.567271e+01,1.192339e+00,1.0,1.187721e+00,2.523979e+01,1.212780e+00,2.523979e+01,1.192266e+00
std,6.252631e+03,1.756189e+01,6.922187e+00,1.113889e+00,1.671654e+01,0.0,8.558413e-01,1.940796e+01,1.097582e+00,1.940796e+01,...,1.789350e+01,1.113742e+00,1.789350e+01,8.947432e-01,0.0,8.700631e-01,1.754363e+01,1.119242e+00,1.754363e+01,9.007319e-01
min,3.000000e+01,2.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00,1.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
25%,5.381500e+03,1.600000e+01,1.575000e+01,0.000000e+00,3.700000e+01,1.0,1.000000e+00,1.200000e+01,1.000000e+00,1.200000e+01,...,1.200000e+01,1.000000e+00,1.200000e+01,1.000000e+00,1.0,1.000000e+00,1.100000e+01,1.000000e+00,1.100000e+01,1.000000e+00
50%,1.120300e+04,3.450000e+01,2.150000e+01,0.000000e+00,4.300000e+01,1.0,1.000000e+00,2.300000e+01,1.000000e+00,2.300000e+01,...,2.100000e+01,1.000000e+00,2.100000e+01,1.000000e+00,1.0,1.000000e+00,2.100000e+01,1.000000e+00,2.100000e+01,1.000000e+00
75%,1.607150e+04,4.700000e+01,2.725000e+01,0.000000e+00,5.800000e+01,1.0,1.000000e+00,3.800000e+01,1.000000e+00,3.800000e+01,...,3.800000e+01,1.000000e+00,3.800000e+01,1.000000e+00,1.0,1.000000e+00,3.700000e+01,1.000000e+00,3.700000e+01,1.000000e+00
max,2.216700e+04,5.900000e+01,3.300000e+01,2.000000e+01,8.300000e+01,1.0,2.100000e+01,1.440000e+02,2.100000e+01,1.440000e+02,...,1.200000e+02,2.100000e+01,1.200000e+02,2.100000e+01,1.0,2.100000e+01,1.200000e+02,2.100000e+01,1.200000e+02,2.100000e+01


In [29]:
X_train, y_train = df_to_X_y(train_set_ranks)

scores = cross_validate(baseline_reg, X_train, y=y_train,
                        cv=tscv.split(train_set_ranks['date_block_num'], 
                                      window=16),
                        verbose=2, scoring='neg_root_mean_squared_error')
scores

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] ................................................. , total=  14.2s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.2s remaining:    0.0s


[CV] ................................................. , total=  13.0s
[CV]  ................................................................
[CV] ................................................. , total=  10.5s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   37.8s finished


{'fit_time': array([13.01769137, 11.79563546,  9.67284179]),
 'score_time': array([1.17500448, 1.21355438, 0.8690455 ]),
 'test_score': array([-0.79122826, -0.92876298, -0.89849347])}

In [30]:
np.mean(scores['test_score']), np.std(scores['test_score'])

(-0.8728282372454572, 0.059008350773385475)

Considering this doesn't include any of the previous features, it's a pretty good score :)